In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [7]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from numpy.linalg import inv

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [8]:
def mat_mul_coo(matrix1, matrix2):
    if matrix1.shape != matrix2.shape:
        raise ValueError("The number of columns in matrix 1 must match the number of rows in matrix 2")
    data = []
    row = []
    col = []
    #I will Convert the COO format of matrix B to CSR format for faster computations.
    matrix2 = matrix2.tocsr()
 
    for i in range(matrix1.shape[0]):
        mat1_row_indices = matrix1.row[matrix1.row == i]
        mat1_row_values = matrix1.data[matrix1.row == i]
        for j in range(matrix2.shape[1]):
            mat2_col_indices = matrix2.indices[matrix2.indptr[j]:matrix2.indptr[j+1]]
            mat2_col_values = matrix2.data[matrix2.indptr[j]:matrix2.indptr[j+1]]
            dot_prod = np.sum(mat1_row_values[mat2_col_indices] * mat2_col_values)
            if dot_prod != 0:
                data.append(dot_prod)
                row.append(i)
                col.append(j)
    C = coo_matrix((data, (row, col)), shape=(matrix1.shape[0], matrix2.shape[1]))
    return C

implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [9]:

def mat_mul_csr(matrix1, matrix2):
    assert matrix1.shape[1] == matrix2.shape[0]

    # Initialize the result matrix with zeros
    C = csr_matrix((matrix1.shape[0], matrix2.shape[1]), dtype=np.float32)
    
    for i in range(matrix1.shape[0]):
        for j in range(matrix2.shape[1]):
            dot_product = 0
            for k in range(matrix1.indptr[i], matrix1.indptr[i+1]):
                dot_product += matrix1.data[k] * matrix2[matrix1.indices[k], j]
            C[i, j] = dot_product
            
    return C       

implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [10]:
#the function of the product of a csr maatrix with a vector
def mat_vec_coo(A, x):

    if A.shape[1] != x.shape[0]:
        raise ValueError("dimension mismatch")
    
    res = np.zeros(x.shape[0], dtype=np.int64)
    for i in range(A.shape[0]):
        for j in range(A.indptr[i],A.indptr[i+1]):
            res[i] += A.data[j] * x[A.indices[j]]
    
    return res

In [11]:
def solve_lin_sys(A,b):
    n = int(input())
    data = []
    indices = []
    indptr = []
    x = []
    A = csr_matrix((data, indices, indptr), shape=(n,n))
    det_A = np.linalg.det(A)

    #we have to check either A is invertible or not
    if det_A != 0:
        print("A is invertible")
        #let's compute the inverse
        A_dense = A.toarray()
        A_inv_dense = inv(A_dense)
        A_inv = csr_matrix(A_inv_dense)
        #to solve Ax=b gives x = A_inv*b
        #it's multiplication of csr matrix with a vector 
        x = mat_vec_coo(A_inv, b)


    else:
        print("A is not invertible")
    return x